# Kaggle Playground - Insurance Premium Regression

## Import libraries

In [1]:
pip install lightautoml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.6/399.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.0/216.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 81.6 MB/s eta 0:00:00
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7591 sha256=a17b9a91ccec3e6b59657d09251adcf859d8cfc5f4a962877f2c43e7c774321c
  Stored in directory: /root/.cache/pip/wheels/e0/d8/b3/6f83a04ab0ec00e691de794d108286bb0f8bcdf4ade19afb57
Successfully built json2html
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.14.3
    Uninstalling statsmodels-0.14.3:
      Successfully uninstalled statsmodels-0.14.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import torch
import os

In [3]:
import pandas as pd              # For data manipulation and analysis
import numpy as np               # For numerical computing
from datetime import datetime
import scipy.stats as stats      # For statistical analysis
import math
import matplotlib                # For plotting and visualization
import matplotlib.pyplot as plt  
from pandas.plotting import parallel_coordinates
import seaborn as sns            # For statistical data visualization
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

## Import datasets

In [4]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')

## Feature Engineering

In [5]:
def extract_date_components(df, date_column):
    # Convert the column to datetime type
    df[date_column] = pd.to_datetime(df[date_column])
    # Extract Year, Month, and Day
    df['Year'] = df[date_column].dt.year
    df['Month'] = df[date_column].dt.month
    df['Day'] = df[date_column].dt.day
    # Drop the original column
    df.drop(columns=[date_column], inplace=True)
    return df

def log_transform(df):
    df['Log Annual Income'] = np.log(df['Annual Income']+1)
    # Drop the original column
    df.drop(columns=['Annual Income'], inplace=True)
    return df

for df in [df_train, df_test]:
    extract_date_components(df, 'Policy Start Date')
    log_transform(df)

In [6]:
num_cols = [col for col in df_test.columns if df_test[col].dtypes in ['float', 'int32']]
print('Numerical Features \n', num_cols)
cat_cols = [col for col in df_test.columns if df_test[col].dtypes in ['object']]
print('Categorical Features \n', cat_cols)

Numerical Features 
 ['Age', 'Number of Dependents', 'Health Score', 'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration', 'Year', 'Month', 'Day', 'Log Annual Income']
Categorical Features 
 ['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location', 'Policy Type', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type']


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer
# Define preprocessing pipelines
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('convert_to_float32', FunctionTransformer(lambda x: x.astype(np.float32)))
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', OrdinalEncoder())
])

# Combine the numerical and categorical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, num_cols),
        ('cat', categorical_pipeline, cat_cols)
    ]
)

preprocessor.set_output(transform="pandas")

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler()),
                                                 ('convert_to_float32',
                                                  FunctionTransformer(func=<function <lambda> at 0x7cba438a9240>))]),
                                 ['Age', 'Number of Dependents', 'Health Score',
                                  'Previous Claims', 'Vehicle Age',
                                  'Credit Score', 'Insurance Duration', 'Year',
                                  'Month', 'Day', 'Log Annual Income']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 ['Gender', 'Marital Status', 'Education Level',
                                  'Occupation', 'Location', 'Policy Type',
                                  'Customer Feedback', 'Smoking Status',
                                  'Exercise Frequency', 'Property Type'])])

In [8]:
# Apply the transformations to the training and validation sets
df_train_processed = preprocessor.fit_transform(df_train)
df_test_processed = preprocessor.transform(df_test)

## Log-transform Target - *Premium Amount*

In [9]:
df_train_automl = pd.concat([df_train_processed, df_train['Premium Amount']], axis=1)
df_train_automl

,num__Age,num__Number of Dependents,num__Health Score,num__Previous Claims,num__Vehicle Age,num__Credit Score,num__Insurance Duration,num__Year,num__Month,num__Day,...,cat__Marital Status,cat__Education Level,cat__Occupation,cat__Location,cat__Policy Type,cat__Customer Feedback,cat__Smoking Status,cat__Exercise Frequency,cat__Property Type,Premium Amount
0,-1.648301,-0.746862,-0.249610,1.216739,1.286338,-1.567375,-0.007023,0.934365,1.604378,0.827741,...,1.0,0.0,1.0,2.0,2.0,2.0,0.0,3.0,2.0,2869.0
1,-0.159542,0.733500,-0.844110,-0.002284,0.420713,0.714630,-1.163391,0.934365,-0.143523,-0.420596,...,0.0,2.0,3.0,0.0,1.0,0.0,1.0,1.0,2.0,1483.0
2,-1.350549,0.733500,1.829212,-0.002284,0.766963,0.013020,-0.777935,0.934365,0.730427,1.622137,...,0.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,2.0,567.0
3,-1.499425,-0.006681,-1.235840,-0.002284,-1.656788,-1.602810,-1.548847,1.611868,-0.143523,-0.420596,...,1.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,765.0
4,-1.499425,-0.746862,-0.437598,-1.221307,-0.271787,0.034281,-0.392479,-0.420641,1.604378,-1.668934,...,2.0,0.0,1.0,0.0,2.0,2.0,1.0,3.0,2.0,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,-0.382856,-1.487043,-0.996082,-0.002284,-0.791163,-1.567375,-0.777935,0.934365,-0.434840,-1.441963,...,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,1303.0
1199996,0.957026,-0.006681,-1.189716,-0.002284,0.074463,0.027194,-0.392479,0.256862,0.730427,-0.647567,...,0.0,2.0,1.0,0.0,1.0,2.0,0.0,3.0,0.0,821.0
1199997,-1.648301,-1.487043,-0.915601,-1.221307,1.632588,0.013020,0.378433,-0.420641,-0.434840,1.054711,...,0.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,1.0,371.0
1199998,1.031464,-0.746862,-0.592267,-0.002284,-0.444912,-1.319331,-0.392479,-0.420641,0.730427,0.373800,...,2.0,3.0,3.0,1.0,2.0,2.0,0.0,0.0,0.0,596.0


In [10]:
df_train_automl['Premium Amount'] = np.log(df_train_automl['Premium Amount'])
df_train_automl['Premium Amount']

0          7.961719
1          7.301822
2          6.340359
3          6.639876
4          7.611842
             ...   
1199995    7.172425
1199996    6.710523
1199997    5.916202
1199998    6.390241
1199999    7.816014
Name: Premium Amount, Length: 1200000, dtype: float64

## LightAutoML

In [11]:
def map_class(x, task, reader):
    if task.name == 'multiclass':
        return reader[x]
    else:
        return x

mapped = np.vectorize(map_class)

def score(task, y_true, y_pred):
    if task.name == 'binary':
        return roc_auc_score(y_true, y_pred)
    elif task.name == 'multiclass':
        return accuracy_score(y_true, np.argmax(y_pred, 1))
    elif task.name == 'reg' or task.name == 'multi:reg':
        return median_absolute_error(y_true, y_pred)
    else:
        raise 'Task is not correct.'
        
def take_pred_from_task(pred, task):
    if task.name == 'binary' or task.name == 'reg':
        return pred[:, 0]
    elif task.name == 'multiclass' or task.name == 'multi:reg':
        return pred
    else:
        raise 'Task is not correct.'
        
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"

In [12]:
RANDOM_STATE = 42
N_THREADS = os.cpu_count()
TIMEOUT = 9 * 3600
N_FOLDS = 10
TARGET = 'Premium Amount'
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [13]:
# task = Task('reg') 
# automl = TabularAutoML(
#     task = task, 
#     timeout = 9 * 3600,
#     cpu_limit = os.cpu_count(),
#     nn_params = {
#     'stop_by_metric': True,
#     'verbose_bar': True},
#     nn_pipeline_params = {"use_qnt": False, "use_te": False},
#     reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE, 'advanced_roles': True}
# )

In [14]:
task = Task('reg') 
automl = TabularAutoML(
    task = task,
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {"use_algos": [["nn"]]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
    nn_params = {
        "n_epochs": 10, 
        "bs": 1024, 
        "num_workers": 0, 
        "path_to_save": None, 
        "freeze_defaults": True,
        "cont_embedder": 'plr',
        'cat_embedder': 'weighted',
        "hidden_size": 128,
        'hid_factor': [4, 4],
        'block_config': [4, 4],
        'embedding_size': 64, 
        'stop_by_metric': True,
        'verbose_bar': True,
        "snap_params": { 'k': 2, 'early_stopping': True, 'patience': 2, 'swa': True}
},
    nn_pipeline_params = {"use_qnt": True, "use_te": False},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

In [15]:
out_of_fold_predictions = automl.fit_predict(
    df_train_automl,
    roles = {
        'target': 'Premium Amount',
        'drop': 'id'
    }, 
    verbose = 3
)

[18:14:44] Stdout logging level is INFO3.
[18:14:44] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[18:14:44] Task: reg

[18:14:44] Start automl preset with listed constraints:
[18:14:44] - time: 32400.00 seconds
[18:14:44] - CPU: 4 cores
[18:14:44] - memory: 16 GB

[18:14:44] Train data shape: (1200000, 22)

[18:14:54] Feats was rejected during automatic roles guess: []
[18:14:54] Layer 1 train process start. Time left 32389.38 secs
[18:15:03] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 ...
[18:15:04] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 =====


val: 100%|██████████| 118/118 [00:05<00:00, 22.67it/s]


[18:16:59] Epoch: 0, train loss: 1.1706992387771606, val loss: 1.1590420007705688, val metric: -1.157461166381836


val: 100%|██████████| 118/118 [00:03<00:00, 32.02it/s]

[18:18:56] Epoch: 1, train loss: 1.1559059619903564, val loss: 1.154036521911621, val metric: -1.1524152755737305


...


val: 100%|██████████| 118/118 [00:03<00:00, 31.84it/s]


[21:50:23] Epoch: 9, train loss: 1.1344821453094482, val loss: 1.1369518041610718, val metric: -1.1370896100997925


val: 100%|██████████| 118/118 [00:03<00:00, 32.80it/s]

[21:50:27] Early stopping: val loss: 1.137054681777954, val metric: -1.1371445655822754


[21:50:27] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 finished. score = -1.127900059880711
[21:50:27] Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 fitting and predicting completed
[21:50:27] Time left 19456.87 secs

[21:50:27] Layer 1 training completed.

[21:50:27] Automl preset training completed in 12943.13 seconds

[21:50:27] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (10 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0) 



In [16]:
from sklearn.metrics import mean_squared_error
print(f'OOF score: {mean_squared_error(df_train_automl[TARGET].values, out_of_fold_predictions.data[:, 0])}') #1.1702855470203595

OOF score: 1.127900059880711


## Prediction

In [17]:
y_test_automl = automl.predict(df_test_processed).data[:, 0]
y_test_automl = np.expm1(y_test_automl)

test: 100%|██████████| 782/782 [00:16<00:00, 47.24it/s]


## Ensemble

In [18]:
y_sub_1 = pd.read_csv('/kaggle/input/rid-train-h2o/submission.csv')
y_sub_2 = pd.read_csv('/kaggle/input/p04e12-blended-submission/submission.csv')
y_sub_3 = pd.read_csv('/kaggle/input/push-the-limit-of-blended-submission/submission.csv')

In [19]:
y_test_ensemble = y_sub_1.copy()
y_test_ensemble['Premium Amount'] = (
    (0/10) * y_sub_1['Premium Amount'] +
    (0/10) * y_sub_2['Premium Amount'] +
    (0/10) * y_sub_3['Premium Amount'] +
    (10/10) * y_test_automl
)

## Submission

In [20]:
df_sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
df_sub['Premium Amount'] = y_test_ensemble['Premium Amount']
df_sub.to_csv('submission_automl.csv', index=False)
df_sub.head()

,id,Premium Amount
0,1200000,858.637451
1,1200001,788.305969
2,1200002,823.157532
3,1200003,793.706055
4,1200004,752.611145
